In [1]:
import cv2
import dlib
import face_recognition
import numpy as np
import pyrealsense2 as rs
from geopy.geocoders import Nominatim
import socket
import time  # Import the time module for adding delays

# Load pre-trained models
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("E:\\face recogniation\\shape_predictor_68_face_landmarks (2).dat\\shape_predictor_68_face_landmarks (2).dat")
face_recognizer = dlib.face_recognition_model_v1("E:\\face recogniation\\dlib_face_recognition_resnet_model_v1.dat\\dlib_face_recognition_resnet_model_v1.dat")

# Load a sample image to search for
search_image = face_recognition.load_image_file("C:\\Users\\saini\\OneDrive\\Pictures\\Screenshots\\1.jpg")
search_face_descriptor = face_recognition.face_encodings(search_image)[0]

# Configure the RealSense camera
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
pipeline.start(config)

# Function to get system geo coordinates with a delay
def get_system_geotags():
    try:
        hostname = socket.gethostname()
        ip_address = socket.gethostbyname(hostname)

        # Add a delay before making the geocoding request
        time.sleep(10)

        geolocator = Nominatim(user_agent="system_location_app")
        location = geolocator.geocode(ip_address)

        if location:
            return location.latitude, location.longitude
        else:
            return None
    except Exception as e:
        print(f"Error: {e}")
        return None

while True:
    frames = pipeline.wait_for_frames()
    color_frame = frames.get_color_frame()

    if not color_frame:
        continue

    frame = np.asanyarray(color_frame.get_data())
    faces = detector(frame)

    for face in faces:
        shape = predictor(frame, face)
        face_descriptor_dlib = face_recognizer.compute_face_descriptor(frame, shape)
        face_descriptor_numpy = np.array(face_descriptor_dlib)

        # Get system geo coordinates
        system_geotags = get_system_geotags()

        if system_geotags:
            # Compare the face descriptor with the search image
            match = face_recognition.compare_faces([search_face_descriptor], face_descriptor_numpy, tolerance=0.6)[0]

            if match:
                (x, y, w, h) = (face.left(), face.top(), face.width(), face.height())
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

                # Print or use the system's geo coordinates
                print(f"System Geotags: Latitude {system_geotags[0]}, Longitude {system_geotags[1]}")

    cv2.imshow("RealSense Camera Search", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

pipeline.stop()
cv2.destroyAllWindows()


RuntimeError: No device connected

In [3]:
from geopy.geocoders import Nominatim
import socket

def get_pc_geolocation():
    try:
        # Get the PC's IP address
        hostname = socket.gethostname()
        ip_address = socket.gethostbyname(hostname)

        # Use Nominatim geocoder to obtain geolocation from IP address
        geolocator = Nominatim(user_agent="pc_location_app")
        location = geolocator.geocode(ip_address)

        if location:
            return location.latitude, location.longitude
        else:
            return None
    except Exception as e:
        print(f"Error: {e}")
        return None

# Example usage
pc_geolocation = get_pc_geolocation()
if pc_geolocation:
    print(f"PC Geolocation: Latitude {pc_geolocation[0]}, Longitude {pc_geolocation[1]}")
else:
    print("Unable to determine PC geolocation.")


Error: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=192.168.1.3&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
Unable to determine PC geolocation.


In [1]:
import cv2
import dlib
import face_recognition
import numpy as np
import pyrealsense2 as rs
from geopy.geocoders import Nominatim
import socket
import time

# Load pre-trained models
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("E:\\face recogniation\\shape_predictor_68_face_landmarks (2).dat\\shape_predictor_68_face_landmarks (2).dat")
face_recognizer = dlib.face_recognition_model_v1("E:\\face recogniation\\dlib_face_recognition_resnet_model_v1.dat\\dlib_face_recognition_resnet_model_v1.dat")

# Load a sample image to search for
search_image = face_recognition.load_image_file("C:\\Users\\saini\\OneDrive\\Pictures\\Screenshots\\1.jpg")
search_face_descriptor = face_recognition.face_encodings(search_image)[0]

# Camera's geographical coordinates
camera_latitude = 30.343168
camera_longitude = 78.020608

# Configure the RealSense camera
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
pipeline.start(config)

# Function to get person's geo coordinates based on camera's location
def get_person_geotags(camera_coords, face_rect):
    # Use a simple offset based on the assumption of proximity
    offset_latitude = 0.0001  # Adjust this offset based on your scenario
    offset_longitude = 0.0001  # Adjust this offset based on your scenario

    person_latitude = camera_coords[0] + offset_latitude
    person_longitude = camera_coords[1] + offset_longitude

    return person_latitude, person_longitude

while True:
    frames = pipeline.wait_for_frames()
    color_frame = frames.get_color_frame()

    if not color_frame:
        continue

    frame = np.asanyarray(color_frame.get_data())
    faces = detector(frame)

    for face in faces:
        shape = predictor(frame, face)
        face_descriptor_dlib = face_recognizer.compute_face_descriptor(frame, shape)
        face_descriptor_numpy = np.array(face_descriptor_dlib)

        # Get person's geo coordinates based on camera's location
        person_geotags = get_person_geotags((camera_latitude, camera_longitude), face)

        # Compare the face descriptor with the search image
        match = face_recognition.compare_faces([search_face_descriptor], face_descriptor_numpy, tolerance=0.6)[0]

        if match and person_geotags:
            (x, y, w, h) = (face.left(), face.top(), face.width(), face.height())
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

            # Print or use the person's geo coordinates
            print(f"Person Geotags: Latitude {person_geotags[0]}, Longitude {person_geotags[1]}")

    cv2.imshow("RealSense Camera Search", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

pipeline.stop()
cv2.destroyAllWindows()


Person Geotags: Latitude 30.343268, Longitude 78.020708
Person Geotags: Latitude 30.343268, Longitude 78.020708
Person Geotags: Latitude 30.343268, Longitude 78.020708
Person Geotags: Latitude 30.343268, Longitude 78.020708
Person Geotags: Latitude 30.343268, Longitude 78.020708
Person Geotags: Latitude 30.343268, Longitude 78.020708
Person Geotags: Latitude 30.343268, Longitude 78.020708
Person Geotags: Latitude 30.343268, Longitude 78.020708
Person Geotags: Latitude 30.343268, Longitude 78.020708
Person Geotags: Latitude 30.343268, Longitude 78.020708
Person Geotags: Latitude 30.343268, Longitude 78.020708
Person Geotags: Latitude 30.343268, Longitude 78.020708
Person Geotags: Latitude 30.343268, Longitude 78.020708
Person Geotags: Latitude 30.343268, Longitude 78.020708
Person Geotags: Latitude 30.343268, Longitude 78.020708
Person Geotags: Latitude 30.343268, Longitude 78.020708
Person Geotags: Latitude 30.343268, Longitude 78.020708
Person Geotags: Latitude 30.343268, Longitude 78

Person Geotags: Latitude 30.005974, Longitude 77.924435
Person Geotags: Latitude 30.005866, Longitude 77.924502
Person Geotags: Latitude 30.005328, Longitude 77.924839
Person Geotags: Latitude 30.005221, Longitude 77.924906
Person Geotags: Latitude 30.023734, Longitude 77.923254
Person Geotags: Latitude 30.024151, Longitude 77.923047
Person Geotags: Latitude 30.005866, Longitude 77.924502
Person Geotags: Latitude 30.005866, Longitude 77.924502
Person Geotags: Latitude 30.024429, Longitude 77.922909
Person Geotags: Latitude 30.024429, Longitude 77.922909
Person Geotags: Latitude 30.024012, Longitude 77.923116
Person Geotags: Latitude 30.024290, Longitude 77.922978
Person Geotags: Latitude 30.024707, Longitude 77.922771
Person Geotags: Latitude 30.025771, Longitude 77.923627
Person Geotags: Latitude 30.024707, Longitude 77.922771
Person Geotags: Latitude 30.024568, Longitude 77.922840
Person Geotags: Latitude 30.024846, Longitude 77.922702
Person Geotags: Latitude 30.005651, Longitude 77